In [2]:
import yfinance as yf
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import time

# Función principal para ejecutar el análisis
def analizar_acciones(compra_fecha_str):
    # Mostrar indicador de carga
    display(Markdown("**Procesando datos...**"))

    # Simular procesamiento (para mostrar el indicador de carga)
    time.sleep(1)

    # Obtención de la acción
    infal = yf.Ticker("0P0001MIBG.F")

    # Obtener datos históricos de la acción
    historical_data = infal.history(period="max")

    # Eliminar la zona horaria del índice
    historical_data.index = historical_data.index.tz_localize(None)

    # Convertir la fecha de compra a datetime
    compra_fecha = pd.to_datetime(compra_fecha_str).tz_localize(None)

    # Si la fecha de compra no está en los datos, tomar la fecha más cercana disponible
    if compra_fecha not in historical_data.index:
        # Obtener la fecha más cercana posterior (si no existe la fecha exacta)
        compra_fecha = historical_data.index[historical_data.index.searchsorted(compra_fecha, side='right')]

    # Filtrar los datos desde la fecha de compra
    historical_data['Date'] = historical_data.index
    historical_data = historical_data[historical_data['Date'] >= compra_fecha]

    # Calcular la rentabilidad desde la compra (diferencia entre el precio de cierre en la fecha de compra y el precio más reciente)
    precio_compra = historical_data.iloc[0]['Close']
    precio_ultimo = historical_data.iloc[-1]['Close']
    rentabilidad = (precio_ultimo - precio_compra) / precio_compra * 100

    # Calcular la volatilidad (desviación estándar de los precios de cierre)
    volatilidad = historical_data['Close'].pct_change().std() * (252 ** 0.5)  # Anualizada

    # Calcular la máxima caída (drawdown)
    historical_data['Rolling_max'] = historical_data['Close'].cummax()
    historical_data['Drawdown'] = historical_data['Close'] / historical_data['Rolling_max'] - 1
    max_drawdown = historical_data['Drawdown'].min()

    # Limpiar la salida antes de mostrar los resultados
    clear_output()

    # Mostrar los resultados con Markdown y colores
    resultado = f"""
    # Resultados desde la fecha de compra ({compra_fecha.date()}):

    **Rentabilidad:** {rentabilidad:.2f}%
    **Volatilidad anualizada:** {volatilidad:.2f}
    **Máxima caída:** {max_drawdown * 100:.2f}%
    """
    display(Markdown(resultado))

# Crear un formulario de entrada para la fecha de compra
fecha_widget = widgets.DatePicker(
    description='Fecha de compra',
    disabled=False
)

# Mensaje de error si no se ha seleccionado una fecha válida
mensaje_error = widgets.Label(value="")

# Botón para ejecutar el análisis
boton = widgets.Button(description="Analizar acción", button_style='success')

# Función para ejecutar al presionar el botón
def on_button_click(b):
    # Si no se selecciona una fecha
    if fecha_widget.value is None:
        mensaje_error.value = "**¡Por favor, selecciona una fecha válida!**"
        display(mensaje_error)
    else:
        mensaje_error.value = ""
        fecha_str = str(fecha_widget.value)
        analizar_acciones(fecha_str)

boton.on_click(on_button_click)

# Mostrar widgets
display(Markdown("### Bienvenido a la herramienta de análisis de acciones!"))
display(Markdown("Selecciona la fecha en la que compraste las acciones para ver el análisis de rentabilidad, volatilidad y caída máxima."))
display(fecha_widget, boton)


    # Resultados desde la fecha de compra (2024-01-02):
    
    **Rentabilidad:** 17.94%  
    **Volatilidad anualizada:** 0.08  
    **Máxima caída:** -5.47%
    